In [1]:
import pandas as pd 

df = pd.read_csv("academy_awards.csv", encoding="ISO-8859-1 ")

let's look at the data and see  if we can spot any quality issues 

In [2]:
df.head(3)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN


We can see there are 6 unnamed columns at the end. 

Will use the value_counts method to explore if any of them have valid values that we need.

Also  notice that the Additional Info column contains a few different formatting styles.

We will need to clean this column up.

In [3]:
df.iloc[:,5].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

*                                                                                                               7
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 resilience                                                                                                     1
 discoverer of stars                                                                                            1
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
Name: Unnamed: 5, dtype: int64

In [4]:
df.iloc[:,7].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

 while requiring no dangerous solvents. [Systems]"    1
*                                                     1
 kindly                                               1
Name: Unnamed: 7, dtype: int64

The dataset is incredibly messy, most columns don't have consistent formatting, which is incredibly important when we use SQL to query the data later on. Other columns vary in the information they convey based on the type of awards category that row corresponds to

<h1>Filtering the data</h1>

In [5]:
df["Year"].head(2)

0    2010 (83rd)
1    2010 (83rd)
Name: Year, dtype: object

Before we filter the data, let's clean up the Year column by selecting just the first 4 digits in each value in the column, therefore excluding the value in parentheses

In [6]:
# As you can see on cell 89 year type is object let's convert the Year column to the int64 data type using astype
df["Year"] = df["Year"].str[0:4].astype("int64")

In [7]:
df["Year"].head(2)

0    2010
1    2010
Name: Year, dtype: int64

In [8]:
#select only the rows from the Dataframe where the Year column is larger than 2000
later_than_2000 = df[df["Year"] > 2000]

In [9]:
# lets fillter award_categories that we're interested in such as Art Direction
later_than_2000["Category"][later_than_2000["Category"]=="Art Direction"].head(3)

23    Art Direction
24    Art Direction
25    Art Direction
Name: Category, dtype: object

In [10]:
award_categories = ["Actor -- Leading Role", "Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]

In [11]:
# select only  where the Category matches one of the 4 awards we're interested in
nominations  = later_than_2000[later_than_2000["Category"].isin(award_categories) ]

In [12]:
# now in nominations we dont have Art Direction catg
nominations["Category"][nominations["Category"]=="Art Direction"].head(3)

Series([], Name: Category, dtype: object)

now let's convert the "Won?"  column to reflect this. Also rename the Won? column to Won so that it's consistent with the other column names. Finally, get rid of the 6 extra, unnamed columns, since they contain only null values in our filtered Dataframe nominations.

In [13]:
nominations['Won?'].head(4)

0     NO
1     NO
2     NO
3    YES
Name: Won?, dtype: object

In [14]:
nominations.iloc[:,5].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)
nominations.iloc[:,7].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

Series([], Name: Unnamed: 7, dtype: int64)

In [15]:
#the 6 extra, unnamed columns contain only null values in our filtered Dataframe nominations so lets get rid of them
nominations.iloc[:,9].value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

Series([], Name: Unnamed: 9, dtype: int64)

column "Won?" is value is YES or NO  lets convert it to 1 or 0  
Also will rename the Won? column to Won  o that it's consistent with the other column names

In [16]:
#using Series method map to replace all NO values with 0 and all YES values with 1
replace_d = { 'YES': 1, 'NO': 0 } 
#reassign 
nominations['Won?'] = nominations['Won?'].map(replace_d)

C:\Users\mebkar\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [17]:
#verify
nominations['Won?'].head(4)

0    0
1    0
2    0
3    1
Name: Won?, dtype: int64

In [18]:
nominations['Won'] = nominations['Won?']

C:\Users\mebkar\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
nominations['Won'].head(4)

0    0
1    0
2    0
3    1
Name: Won, dtype: int64

In [20]:
#now lets drop the extraneous columns
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)

<h1>cleanup columns</h1>

now lets cleanup Additional Info column, whose values are formatted like so:

MOVIE {'CHARACTER'}

Here are some examples:

Biutiful {'Uxbal'} - Biutiful is the movie and Uxbal is the character this nominee played.

In [21]:
final_nominations["Additional Info"].head(4)

0                        Biutiful {'Uxbal'}
1             True Grit {'Rooster Cogburn'}
2    The Social Network {'Mark Zuckerberg'}
3      The King's Speech {'King George VI'}
Name: Additional Info, dtype: object

In [22]:
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")

In [23]:
additional_info_two = additional_info_one.str.split(" {'")

In [24]:
movie_names = additional_info_two.str[0]

In [25]:
characters =  additional_info_two.str[1]

In [26]:
final_nominations["Movie"] = movie_names

In [27]:
final_nominations["Character"] = characters

In [28]:
final_nominations = final_nominations.drop("Additional Info", axis=1)
final_nominations.head(5)

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


<b>Now that our Dataframe is cleaned up, let's write these records to a SQL database
</b>

we will create SQLite database nominations.db and connect to it 

<h1>Exporting to SQLite</h1>

In [35]:
import sqlite3 
conn = sqlite3.connect("nominations.db")

In [36]:
final_nominations.to_sql("nominations", conn, index=False)

<h1>Verifying in SQL </h1>

In [31]:
query_one = "pragma table_info(nominations);"
query_two = "select * from nominations limit 10;"
print(conn.execute(query_one).fetchall())


[(0, 'index', 'INTEGER', 0, None, 0), (1, 'Year', 'INTEGER', 0, None, 0), (2, 'Category', 'TEXT', 0, None, 0), (3, 'Nominee', 'TEXT', 0, None, 0), (4, 'Won', 'INTEGER', 0, None, 0), (5, 'Movie', 'TEXT', 0, None, 0), (6, 'Character', 'TEXT', 0, None, 0)]


In [32]:
print(conn.execute(query_two).fetchall())

conn.close()

[(0, 2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal'), (1, 2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn'), (2, 2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg'), (3, 2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI'), (4, 2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston'), (5, 2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund'), (6, 2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop'), (7, 2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin'), (8, 2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul'), (9, 2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')]


<h3>next step : Creating relations in SQLite<h3>

 In the previous steps we walked through how to clean and prepare the original CSV dataset on Academy Award nominations and export the data into a SQLite database as a single, denormalized table. 
 
Now we will walk through how to normalize our single table into multiple tables and how to create relations between them.

In [39]:
conn1 = sqlite3.connect("nominations.db")

In [40]:

schema = conn1.execute("pragma table_info(nominations);").fetchall()
first_ten = conn1.execute("select * from nominations limit 10;").fetchall()

for r in schema:
    print(r)
    
for r in first_ten:
    print(r)

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'Year', 'INTEGER', 0, None, 0)
(2, 'Category', 'TEXT', 0, None, 0)
(3, 'Nominee', 'TEXT', 0, None, 0)
(4, 'Won', 'INTEGER', 0, None, 0)
(5, 'Movie', 'TEXT', 0, None, 0)
(6, 'Character', 'TEXT', 0, None, 0)
(0, 2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(1, 2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2, 2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(3, 2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(4, 2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(5, 2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(6, 2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(7, 2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(8, 2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0,

Let's now add information on the host for each awards ceremony. Instead of adding a Host column to the nominations table and having lots of redundant data, we'll create a separate table called ceremonies which contains data specific to the ceremony itself

<h1>Creating the ceremonies table

In [43]:
create_ceremonies = "create table ceremonies (id integer primary key, year integer, host text);"
conn1.execute(create_ceremonies)

In [47]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [45]:
insert_query2 = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"
conn1.executemany(insert_query2, years_hosts)

In [49]:
print(conn1.execute("select * from ceremonies limit 10;").fetchall())

[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]


In [50]:
print(conn1.execute("pragma table_info(ceremonies);").fetchall())

[(0, 'id', 'integer', 0, None, 1), (1, 'year', 'integer', 0, None, 0), (2, 'host', 'text', 0, None, 0)]


<h1>Foreign key constraints

In [51]:
conn1.execute("PRAGMA foreign_keys = ON;")

<h2>Creating relations in SQLite

The next step is to remove the Year column from nominations and add a new column, ceremony_id, that contains the foreign key reference to the id column in the ceremonies table. Unfortunately, we can't remove columns from an existing table in SQLite

In [57]:
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won integer,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

In [59]:
conn1.execute(create_nominations_two)

In [64]:
nominations_two_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''

In [65]:
joined_nominations = conn1.execute(nominations_two_query).fetchall()

In [66]:
insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

In [68]:
conn1.executemany(insert_nominations_two, joined_nominations)

In [70]:
print(conn1.execute("select * from nominations_two limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)]


<h2>Deleting and renaming tables

In [71]:
 conn1.execute("DROP TABLE nominations")

In [72]:
 conn1.execute(" ALTER TABLE nominations_two RENAME TO nominations")

<h2> Creating a join table </h2>

we will create 3 tables we need to model the relationship between movies and actors. 

so we need to create the movies and actors tables before creating the movies_actors table for the foreign key references to work

In [79]:
movie_Q = '''CREATE TABLE movies (
id INTEGER PRIMARY KEY,
movie text 
);'''

conn1.execute(movie_Q)

In [80]:
actor_Q = '''CREATE TABLE actors (
id INTEGER PRIMARY KEY,
actor text 
);'''

conn1.execute(actor_Q)

In [81]:
movies_actors_Q = '''CREATE TABLE movies_actors (
id INTEGER PRIMARY KEY,
movie_id INTEGER REFERENCES movies(id), 
actor_id INTEGER REFERENCES actors(id)
);'''

conn1.execute(movies_actors_Q)


<h3>Populating the movies and actors tables

In [85]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn1.execute(insert_movies)
conn1.execute(insert_actors)

print(conn1.execute("select * from movies limit 5;").fetchall())
print(conn1.execute("select * from actors limit 5;").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


<h3>Populating a join table

In [87]:
pairs_query = "select movie,nominee from nominations;"
movie_actor_pairs = conn.execute(pairs_query).fetchall()

join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),(select id from actors where actor == ?));"
conn1.executemany(join_table_insert,movie_actor_pairs)

print(conn1.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]
